# Base

In [ ]:
import os
import numpy as np 
import pywt
import json

from numpy import genfromtxt

import scipy

Mounted at /content/gdrive


In [ ]:
DIR_RAW_DATA="/content/data/" #Path to your rawdata txt file, this means that t1_experimental.txt,t1_control.txt,...t4_experimental.txt,t4_control.txt
DIR_TREATED_DATA="/content/data_treated/" #Path where you will store your treated data

# Data Extraction

In [ ]:
def wavelet(signal):
  """
  This functions extract 40s sample of the 80s signal with a sliding window (40s and step of 1s)
  Then compute the wavelet transform on each extract of 40s extract (50Hz signal ==> length = 2000) 
  The output is a 63-size vector
  """
  L_features=[]
  for k in range(0,2001,50):
    Ar = pywt.downcoef("a", signal[k:k+2000], 'haar', mode='symmetric', level=5)
    L_features.append(Ar.tolist())
  return L_features

In [ ]:
"""
This cells extract the mean of the 80s baseline signal for each subject (for experimental group and for control group) (t1 is baseline = driving normally)
"""


f = open(DIR_RAW_DATA+"t1_experimental.txt", "r")
mean_base_exp=[]
for i in range (0,16) :
  a=[]
  strr=f.readline().split("\t")
  strr[-1]=strr[-1][0:-1]
  mean_base_exp.append(np.mean(list(map(float,strr))))
f.close ()

mean_base_cont=[]
f = open(DIR_RAW_DATA+"t1_control.txt", "r")
for i in range (0,13) :
  a=[]
  strr=f.readline().split("\t")
  strr[-1]=strr[-1][0:-1]
  mean_base_cont.append(np.mean(list(map(float,strr))))
f.close ()

In [ ]:
""" 
This cells opens each rawdata file. For each file and for each subject the 80s signal (for each trial) is divided by the mean of the baseline signal of the same subject.
As pupil signal can be very different from people to people, the idea is to compute a ratio with the baseline for each subject.
Once this ratio is computed, the divided signal is splitted in extract with a sliding window and Wavelet transform is applied

For each file oppened a json file is created to store the data
"""


l= os.listdir(DIR_RAW_DATA)

for name in l:
  L=[]
  group=name[3:-4]
  f = open(DIR_RAW_DATA"+name, "r")
  result_final=[]
  if group=="control":
    n=13
    for i in range (0,n) :
      a=[]
      strr=f.readline().split("\t")
      strr[-1]=strr[-1][0:-1]
      L.append(list(map(float,strr)))
      features=wavelet(np.array(L[i])/features_base_cont[i])
      result_final.append(features)
  if group=="experimental":
    n=16
    for i in range (0,n) :
      a=[]
      strr=f.readline().split("\t")
      strr[-1]=strr[-1][0:-1]
      L.append(list(map(float,strr)))
      features=wavelet(np.array(L[i])/features_base_exp[i])
      result_final.append(features)
  f.close()
  print(len(result_final))
  with open(DIR_TREATED_DATA+"data_ext_"+name+".json", 'w') as f:
    json.dump(result_final, f)